In [1]:
import data.read_data as read
import data.transform_data as transform


In [2]:
train_csv_file_name = "res/train.csv"
test_csv_file_name = "res/test.csv"
test_label_csv_file_name = "res/test_labels.csv"
submission_csv_file_name = "res/sample_submission.csv"


In [3]:
df = read.load_train_csv(train_csv_file_name)


In [4]:
print(df)


                      id                                       comment_text  \
0       0000997932d777bf  Explanation\nWhy the edits made under my usern...   
1       000103f0d9cfb60f  D'aww! He matches this background colour I'm s...   
2       000113f07ec002fd  Hey man, I'm really not trying to edit war. It...   
3       0001b41b1c6bb37e  "\nMore\nI can't make any real suggestions on ...   
4       0001d958c54c6e35  You, sir, are my hero. Any chance you remember...   
5       00025465d4725e87  "\n\nCongratulations from me as well, use the ...   
6       0002bcb3da6cb337       COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK   
7       00031b1e95af7921  Your vandalism to the Matt Shirvington article...   
8       00037261f536c51d  Sorry if the word 'nonsense' was offensive to ...   
9       00040093b2687caa  alignment on this subject and which are contra...   
10      0005300084f90edc  "\nFair use rationale for Image:Wonju.jpg\n\nT...   
11      00054a5e18b50dd4  bbq \n\nbe a man and lets 

In [5]:
print(df.dtypes)


id               object
comment_text     object
toxic             int64
severe_toxic      int64
obscene           int64
threat            int64
insult            int64
identity_hate     int64
dtype: object


In [6]:
X_train = df.iloc[:, :2].values
Y_train = df.iloc[:, 2:].values

X_train = transform.split_comment(X_train)
count, vocab = transform.mk_vocab(X_train)


In [7]:
print(len(vocab))


32015


In [8]:
print(X_train[0])


('0000997932d777bf', array(['Explanation', 'Why', 'the', 'edits', 'made', 'under', 'my',
       'username', 'Hardcore', 'Metallica', 'Fan', 'were', 'reverted',
       '?', 'They', 'weren', "'", 't', 'vandalisms', ',', 'just',
       'closure', 'on', 'some', 'GAs', 'after', 'I', 'voted', 'at', 'New',
       'York', 'Dolls', 'FAC', '.', 'And', 'please', 'don', "'", 't',
       'remove', 'the', 'template', 'from', 'the', 'talk', 'page',
       'since', 'I', "'", 'm', 'retired', 'now', '.', '89', '.', '205',
       '.', '38', '.', '27'], dtype='<U11'))


In [9]:
ident, X = transform.pass_data_to_word_idx(X_train, vocab, count)


In [10]:
print(X)


[array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 16, 17, 37,  2, 38, 39,  2, 40, 41, 42, 26, 16, 43, 44,
       45, 33, 46, 33, 47, 33, 48, 33, 49])
 array([50, 16, 51, 52, 53, 54, 55, 56, 26, 16, 43, 57, 58, 59, 33, 60, 33,
       40, 61, 62, 19, 63, 64, 19, 65, 66])
 array([67, 68, 19, 26, 16, 43, 69, 70, 71, 72, 73, 74, 33, 75, 16, 76, 20,
       77, 54, 78, 79, 80, 81, 82, 83, 84, 85, 72, 86, 87,  3, 88, 89,  6,
       40, 41, 33, 52, 90, 72, 91, 92, 93,  2, 94, 95,  2, 96, 97, 33])
 ...
 array([22146,    19,  2593,   134,    96,   189,   142,  7601,  8141,
          33, 30841,  4046,    33])
 array([  34,  145, 1535,  409,  145,  199,  956,  139,  317, 1209,   22,
          2,  301,   72,  140,    2,  138,  669,  297,   45,   77,   26,
        408,   28,  145,   33])
 array([  34,   33,   33,   33,   26,   69,   36,   16,   17,  117,  139,
        627,   33,   2

In [11]:
from model.conv_model import ConvModel
import torch as th
import torch.nn as nn
from utils.cuda import use_cuda


In [17]:
model = ConvModel(len(vocab))
loss_fn = nn.BCELoss()

if use_cuda():
    model.cuda()
    loss_fn.cuda()

opt = th.optim.SGD(model.parameters(), lr=1e-3)

# TODO boucle apprentissage
out = model(th.Tensor(X[0]).long().cuda())
print(out.size())

loss = loss_fn(out, th.Tensor(Y_train[0]).unsqueeze(0).cuda())
loss.backward()
opt.step()
print(loss.item())

torch.Size([1, 6])
4.928504943847656
